In [1]:
import pandas as pd
import os
from typing import Optional
from pandas import DataFrame
pd.set_option('display.max_columns', None)

In [2]:
from utils import copy_df
from config import DATA_FOLDER

### Download dos dados da página de orçamento da Prefeitura

In [3]:
BASE_URL = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2024/'

In [4]:
def build_url(base_url:str, file:str)->str:

    return base_url + file

def csv_fpath(fpath:str)->str:

    if not fpath.endswith('.csv'):
        fpath = fpath+'.csv'
    return fpath

def save_csv(df:pd.DataFrame, fpath:str, **save_kwargs)->str:

    fpath = csv_fpath(fpath)

    df.to_csv(fpath, sep=';', index=False, **save_kwargs)

    return os.path.abspath(fpath)


def load_csv(fpath:str, **read_kwargs)->pd.DataFrame:

    if not fpath.endswith('.csv'):
        raise ValueError(f'fpath {fpath} is not a csv file')
    
    return pd.read_csv(fpath, **read_kwargs)


def solve_data_file_path(file:str, folder=DATA_FOLDER)->str:

    if not os.path.exists(folder):
        os.mkdir(folder)

    folder = os.path.abspath(folder)

    if os.path.dirname(file) == folder:
        return file
    
    return os.path.join(folder, file)


def read_file_from_url(file:str, base_url=BASE_URL, **read_kwargs)->pd.DataFrame:

    url = build_url(base_url, file)

    return pd.read_csv(url, **read_kwargs)


In [5]:


def cached_download(file:str, folder= DATA_FOLDER, base_url = BASE_URL, 
                     read_params:Optional[dict]=None, save_params:Optional[dict]=None,
                     load_from_url_params:Optional[dict]=None)->pd.DataFrame:

    read_params = read_params or {}
    save_params = save_params or {}
    load_from_url_params = load_from_url_params or {}

    filepath = solve_data_file_path(file, folder)

    #early return if already saved - only reads
    if os.path.exists(filepath) and os.path.isfile(filepath):
        print(f'Loading file {file}')
        return load_csv(filepath, **read_params)
    

    df: DataFrame = read_file_from_url(file, base_url=base_url, **load_from_url_params)

    save_csv(df, filepath, **save_params)

    return df


In [6]:
file_execucao_2024= 'basedadosexecucao_1224.csv'
file_da_2024 = 'basedadosDA_1224.csv'


In [7]:
load_from_url_params = {'encoding' : 'latin1', 'sep' : ';'}
read_params = {'sep' : ';'}

df_execucao: DataFrame = cached_download(file_execucao_2024, 
                                         load_from_url_params=load_from_url_params,
                                         read_params=read_params)


df_regionalizacao: DataFrame = cached_download(file_da_2024, 
                                         load_from_url_params=load_from_url_params,
                                        read_params=read_params)

Loading file basedadosexecucao_1224.csv
Loading file basedadosDA_1224.csv


/tmp/ipykernel_107522/3213725603.py:25: DtypeWarning: Columns (27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(fpath, **read_kwargs)


In [8]:
df_execucao.head()

,DataInicial,DataFinal,Cd_AnoExecucao,Cd_Exercicio,Cd_Dotacao_Id,Administracao,Cd_Orgao,Sigla_Orgao,Ds_Orgao,Cd_Unidade,Ds_Unidade,Cd_Funcao,Ds_Funcao,Cd_SubFuncao,Ds_SubFuncao,Cd_Programa,Ds_Programa,PA,PAPA,ProjetoAtividade,Ds_Projeto_Atividade,ProjAtiv_Descricao,Cd_Despesa,Ds_Despesa,Categoria_Despesa,Ds_Categoria,Grupo_Despesa,Ds_Grupo,Cd_Modalidade,Ds_Modalidade,Cd_Elemento,Cd_Fonte,Ds_Fonte,COD_EX_FONT_REC,TXT_EX_FONT_REC,COD_DSTN_REC,TXT_DSTN_REC,COD_VINC_REC_PMSP,TXT_VINC_PMSP,COD_TIP_CRED_ORCM,TXT_TIP_CRED_ORCM,COD_RDZD_FONT_REC,TXT_FONT_REC_RDZD,Cd_Nro_Emenda_Dotacao,Vl_Orcado_Ano,Vl_Suplementado,Vl_Reduzido,Vl_SuplementadoLiquido,Vl_SuplementadoEmTramitacao,Vl_ReduzidoEmTramitacao,Vl_Orcado_Atualizado,Vl_Congelado,Vl_Descongelado,Vl_CongeladoLiquido,Disponivel,Vl_ReservadoLiquido,Vl_EmpenhadoLiquido,Vl_Liquidado,Vl_Pago,Saldo_Dotacao,DataExtracao
0,01/01/2024,31/12/2024,2024,2024,169108,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,Fundo Municipal de Desenvolvimento Social,8,Assistência Social,244,Assistência Comunitária,3023,Proteção à População em Situação de Vulnerabil...,Projetos,Projetos,3399,"Ampliação, Reforma e Requalificação de Equipam...","3399 - Ampliação, Reforma e Requalificação de ...",44905100,Obras e Instalações,4,Despesas de Capital,4,INVESTIMENTOS,90,Aplicações Diretas,51,8,Tesouro Municipal - Recursos Vinculados,1,Exercício corrente,759,Recursos vinculados a fundos,1224,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,0,Inicial,80014,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,NaN,1000,0,0,0,0,0,1000,0,0,0,1000,0,0,0,0,1000,18/01/2025
1,01/01/2024,31/12/2024,2024,2024,173723,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,Fundo Municipal de Desenvolvimento Social,10,Saúde,301,Atenção Básica,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Projetos,Projetos,1525,Construção e Implantação de Equipamentos de At...,1525 - Construção e Implantação de Equipamento...,44505200,Equipamentos e Material Permanente,4,Despesas de Capital,4,INVESTIMENTOS,50,Transferências a Instituições Privadas sem fin...,52,8,Tesouro Municipal - Recursos Vinculados,1,Exercício corrente,759,Recursos vinculados a fundos,1224,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,1,Suplementar,80014,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,NaN,0,"1317978,6",0,"1317978,6",0,0,"1317978,6",0,0,0,"1317978,6",1314177,1314177,1314177,1314177,"3801,6",18/01/2025
2,01/01/2024,31/12/2024,2024,2024,171046,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,Fundo Municipal de Desenvolvimento Social,10,Saúde,301,Atenção Básica,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Projetos,Projetos,1525,Construção e Implantação de Equipamentos de At...,1525 - Construção e Implantação de Equipamento...,44905100,Obras e Instalações,4,Despesas de Capital,4,INVESTIMENTOS,90,Aplicações Diretas,51,8,Tesouro Municipal - Recursos Vinculados,1,Exercício corrente,759,Recursos vinculados a fundos,1224,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,0,Inicial,80014,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,NaN,4449303,0,"1317978,6","-1317978,6",0,0,"3131324,4",0,0,0,"3131324,4","3131324,4","3131324,4","2940879,9","2791266,85",0,18/01/2025
3,01/01/2024,31/12/2024,2024,2024,173862,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,Fundo Municipal de Desenvolvimento Social,10,Saúde,301,Atenção Básica,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Projetos,Projetos,1525,Construção e Implantação de Equipamentos de At...,1525 - Construção e Implantação de Equipamento...,44905100,Obras e Instalações,4,Despesas de Capital,4,INVESTIMENTOS,90,Aplicações Diretas,51,8,Tesouro Municipal - Recursos Vinculados,2,Exercícios anteriores,759,Recursos vinculados a fundos,1224,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,1,Suplementar,80063,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,NaN,0,"37258939,15",0,"37258939,15",0,0,"37258939,15",0,0,0,"37258939,15","37222552,12","37222552,12","309

In [9]:
df_regionalizacao.head()

,COD_EMPRESA_PMSP,COD_EMPENHO,ANO_EMPENHO,CÓDIGO_NLP,ANO_LIQUIDAÇÃO,DATA_LIQUIDAÇÃO,CÓDIGO_ÓRGÃO,SIGLA_ÓRGÃO,DESCRIÇÃO_ÓRGÃO,CÓDIGO_UNIDADE,DESCRIÇÃO_UNIDADE,CÓDIGO_FUNÇÃO,DESCRIÇÃO_FUNÇÃO,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,CÓDIGO_PROJ_ATIV,DESCRIÇÃO_PROJ_ATIV,CÓDIGO_CONTA_DESPESA,DESCRIÇÃO_CONTA_DESPESA,CÓDIGO_FONTE,DESCRIÇÃO_FONTE,CÓDIGO_EXERCÍCIO_FONTE,CÓDIGO_DESTINAÇÃO_RECURSO,CÓDIGO_VÍNCULO_PMSP,CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,DATA_EXTRAÇÃO
0,4,197,2024,362,2024,2024-01-30 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3024,Suporte Administrativo,2100,Administração da Unidade,31900700,Contribuição a Entidades Fechadas de Previdência,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1792,03",15/03/2025
1,4,197,2024,893,2024,2024-02-28 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3024,Suporte Administrativo,2100,Administração da Unidade,31900700,Contribuição a Entidades Fechadas de Previdência,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1780,47",15/03/2025
2,4,197,2024,1973,2024,2024-04-05 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3024,Suporte Administrativo,2100,Administração da Unidade,31900700,Contribuição a Entidades Fechadas de Previdência,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1667,19",15/03/2025
3,4,197,2024,2430,2024,2024-04-29 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3024,Suporte Administrativo,2100,Administração da Unidade,31900700,Contribuição a Entidades Fechadas de Previdência,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1667,19",15/03/2025
4,4,2028,2024,3142,2024,2024-05-29 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3024,Suporte Administrativo,2100,Administração da Unidade,31900700,Contribuição a Entidades Fechadas de Previdência,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1782,02",15/03/2025


### Execucao Orçamentária

In [10]:
df_execucao.head()

,DataInicial,DataFinal,Cd_AnoExecucao,Cd_Exercicio,Cd_Dotacao_Id,Administracao,Cd_Orgao,Sigla_Orgao,Ds_Orgao,Cd_Unidade,Ds_Unidade,Cd_Funcao,Ds_Funcao,Cd_SubFuncao,Ds_SubFuncao,Cd_Programa,Ds_Programa,PA,PAPA,ProjetoAtividade,Ds_Projeto_Atividade,ProjAtiv_Descricao,Cd_Despesa,Ds_Despesa,Categoria_Despesa,Ds_Categoria,Grupo_Despesa,Ds_Grupo,Cd_Modalidade,Ds_Modalidade,Cd_Elemento,Cd_Fonte,Ds_Fonte,COD_EX_FONT_REC,TXT_EX_FONT_REC,COD_DSTN_REC,TXT_DSTN_REC,COD_VINC_REC_PMSP,TXT_VINC_PMSP,COD_TIP_CRED_ORCM,TXT_TIP_CRED_ORCM,COD_RDZD_FONT_REC,TXT_FONT_REC_RDZD,Cd_Nro_Emenda_Dotacao,Vl_Orcado_Ano,Vl_Suplementado,Vl_Reduzido,Vl_SuplementadoLiquido,Vl_SuplementadoEmTramitacao,Vl_ReduzidoEmTramitacao,Vl_Orcado_Atualizado,Vl_Congelado,Vl_Descongelado,Vl_CongeladoLiquido,Disponivel,Vl_ReservadoLiquido,Vl_EmpenhadoLiquido,Vl_Liquidado,Vl_Pago,Saldo_Dotacao,DataExtracao
0,01/01/2024,31/12/2024,2024,2024,169108,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,Fundo Municipal de Desenvolvimento Social,8,Assistência Social,244,Assistência Comunitária,3023,Proteção à População em Situação de Vulnerabil...,Projetos,Projetos,3399,"Ampliação, Reforma e Requalificação de Equipam...","3399 - Ampliação, Reforma e Requalificação de ...",44905100,Obras e Instalações,4,Despesas de Capital,4,INVESTIMENTOS,90,Aplicações Diretas,51,8,Tesouro Municipal - Recursos Vinculados,1,Exercício corrente,759,Recursos vinculados a fundos,1224,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,0,Inicial,80014,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,NaN,1000,0,0,0,0,0,1000,0,0,0,1000,0,0,0,0,1000,18/01/2025
1,01/01/2024,31/12/2024,2024,2024,173723,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,Fundo Municipal de Desenvolvimento Social,10,Saúde,301,Atenção Básica,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Projetos,Projetos,1525,Construção e Implantação de Equipamentos de At...,1525 - Construção e Implantação de Equipamento...,44505200,Equipamentos e Material Permanente,4,Despesas de Capital,4,INVESTIMENTOS,50,Transferências a Instituições Privadas sem fin...,52,8,Tesouro Municipal - Recursos Vinculados,1,Exercício corrente,759,Recursos vinculados a fundos,1224,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,1,Suplementar,80014,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,NaN,0,"1317978,6",0,"1317978,6",0,0,"1317978,6",0,0,0,"1317978,6",1314177,1314177,1314177,1314177,"3801,6",18/01/2025
2,01/01/2024,31/12/2024,2024,2024,171046,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,Fundo Municipal de Desenvolvimento Social,10,Saúde,301,Atenção Básica,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Projetos,Projetos,1525,Construção e Implantação de Equipamentos de At...,1525 - Construção e Implantação de Equipamento...,44905100,Obras e Instalações,4,Despesas de Capital,4,INVESTIMENTOS,90,Aplicações Diretas,51,8,Tesouro Municipal - Recursos Vinculados,1,Exercício corrente,759,Recursos vinculados a fundos,1224,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,0,Inicial,80014,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,NaN,4449303,0,"1317978,6","-1317978,6",0,0,"3131324,4",0,0,0,"3131324,4","3131324,4","3131324,4","2940879,9","2791266,85",0,18/01/2025
3,01/01/2024,31/12/2024,2024,2024,173862,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,Fundo Municipal de Desenvolvimento Social,10,Saúde,301,Atenção Básica,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Projetos,Projetos,1525,Construção e Implantação de Equipamentos de At...,1525 - Construção e Implantação de Equipamento...,44905100,Obras e Instalações,4,Despesas de Capital,4,INVESTIMENTOS,90,Aplicações Diretas,51,8,Tesouro Municipal - Recursos Vinculados,2,Exercícios anteriores,759,Recursos vinculados a fundos,1224,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,1,Suplementar,80063,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,NaN,0,"37258939,15",0,"37258939,15",0,0,"37258939,15",0,0,0,"37258939,15","37222552,12","37222552,12","309

Primeiro de tudo vamos filtrar para administração direta que é o que interessa pro estudo.

In [11]:
df_execucao['Administracao'].unique()

array(['Direta', 'Indireta', 'Legislativo'], dtype=object)

In [12]:
#filtrando para ser só administração direta

df_execucao = df_execucao[df_execucao['Administracao']=='Direta']

Como checamos abaixo, é a execução do ano inteiro de 2024.

In [13]:
df_execucao['DataInicial'].unique()

array(['01/01/2024'], dtype=object)

In [14]:
df_execucao['DataFinal'].unique()

array(['31/12/2024'], dtype=object)

In [15]:
df_execucao['Cd_AnoExecucao'].unique()

array([2024])

In [16]:
df_execucao['Cd_Exercicio'].unique()

array([2024])

Vamos filtrar as colunas que nos interessam

In [17]:
for col in df_execucao.columns:
    print(col)

DataInicial
DataFinal
Cd_AnoExecucao
Cd_Exercicio
Cd_Dotacao_Id
Administracao
Cd_Orgao
Sigla_Orgao
Ds_Orgao
Cd_Unidade
Ds_Unidade
Cd_Funcao
Ds_Funcao
Cd_SubFuncao
Ds_SubFuncao
Cd_Programa
Ds_Programa
PA
PAPA
ProjetoAtividade
Ds_Projeto_Atividade
ProjAtiv_Descricao
Cd_Despesa
Ds_Despesa
Categoria_Despesa
Ds_Categoria
Grupo_Despesa
Ds_Grupo
Cd_Modalidade
Ds_Modalidade
Cd_Elemento
Cd_Fonte
Ds_Fonte
COD_EX_FONT_REC
TXT_EX_FONT_REC
COD_DSTN_REC
TXT_DSTN_REC
COD_VINC_REC_PMSP
TXT_VINC_PMSP
COD_TIP_CRED_ORCM
TXT_TIP_CRED_ORCM
COD_RDZD_FONT_REC
TXT_FONT_REC_RDZD
Cd_Nro_Emenda_Dotacao
Vl_Orcado_Ano
Vl_Suplementado
Vl_Reduzido
Vl_SuplementadoLiquido
Vl_SuplementadoEmTramitacao
Vl_ReduzidoEmTramitacao
Vl_Orcado_Atualizado
Vl_Congelado
Vl_Descongelado
Vl_CongeladoLiquido
Disponivel
Vl_ReservadoLiquido
Vl_EmpenhadoLiquido
Vl_Liquidado
Vl_Pago
Saldo_Dotacao
DataExtracao


In [18]:
df_regionalizacao.head()

,COD_EMPRESA_PMSP,COD_EMPENHO,ANO_EMPENHO,CÓDIGO_NLP,ANO_LIQUIDAÇÃO,DATA_LIQUIDAÇÃO,CÓDIGO_ÓRGÃO,SIGLA_ÓRGÃO,DESCRIÇÃO_ÓRGÃO,CÓDIGO_UNIDADE,DESCRIÇÃO_UNIDADE,CÓDIGO_FUNÇÃO,DESCRIÇÃO_FUNÇÃO,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,CÓDIGO_PROJ_ATIV,DESCRIÇÃO_PROJ_ATIV,CÓDIGO_CONTA_DESPESA,DESCRIÇÃO_CONTA_DESPESA,CÓDIGO_FONTE,DESCRIÇÃO_FONTE,CÓDIGO_EXERCÍCIO_FONTE,CÓDIGO_DESTINAÇÃO_RECURSO,CÓDIGO_VÍNCULO_PMSP,CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,DATA_EXTRAÇÃO
0,4,197,2024,362,2024,2024-01-30 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3024,Suporte Administrativo,2100,Administração da Unidade,31900700,Contribuição a Entidades Fechadas de Previdência,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1792,03",15/03/2025
1,4,197,2024,893,2024,2024-02-28 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3024,Suporte Administrativo,2100,Administração da Unidade,31900700,Contribuição a Entidades Fechadas de Previdência,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1780,47",15/03/2025
2,4,197,2024,1973,2024,2024-04-05 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3024,Suporte Administrativo,2100,Administração da Unidade,31900700,Contribuição a Entidades Fechadas de Previdência,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1667,19",15/03/2025
3,4,197,2024,2430,2024,2024-04-29 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3024,Suporte Administrativo,2100,Administração da Unidade,31900700,Contribuição a Entidades Fechadas de Previdência,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1667,19",15/03/2025
4,4,2028,2024,3142,2024,2024-05-29 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3024,Suporte Administrativo,2100,Administração da Unidade,31900700,Contribuição a Entidades Fechadas de Previdência,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1782,02",15/03/2025


In [19]:
codigos_dotacao = [col for col in df_execucao.columns if (col.startswith('Cd')
                   or col in {'ProjetoAtividade', 'Categoria_Despesa'} or col.startswith('COD'))
                   and col not in {'Cd_AnoExecucao', 'Cd_Exercicio', 'Cd_Dotacao_Id', 'Cd_Nro_Emenda_Dotacao',
                                   'Categoria_Despesa', 'Cd_Modalidade', 'Cd_Elemento','COD_RDZD_FONT_REC'}]

In [20]:
codigos_dotacao

['Cd_Orgao',
 'Cd_Unidade',
 'Cd_Funcao',
 'Cd_SubFuncao',
 'Cd_Programa',
 'ProjetoAtividade',
 'Cd_Despesa',
 'Cd_Fonte',
 'COD_EX_FONT_REC',
 'COD_DSTN_REC',
 'COD_VINC_REC_PMSP',
 'COD_TIP_CRED_ORCM']

In [21]:
for col in codigos_dotacao:
    df_execucao[col] = df_execucao[col].astype(int)

In [22]:
@copy_df
def build_dotacao(df:pd.DataFrame, cols_cod_dotacao:list[str]=codigos_dotacao)->DataFrame:

    df_cods = df[cols_cod_dotacao].copy()
    df['dotacao'] = df_cods.astype(str).agg('.'.join, axis=1)

    return df

In [23]:
df_execucao = build_dotacao(df_execucao)

In [24]:
df_execucao['dotacao'].sample(10)

769     16.10.12.365.3025.2877.31901100.4.1.540.9001.0
2043    19.10.27.812.3017.4503.33503900.0.1.501.7044.1
890     16.10.12.368.3010.2831.33903200.0.1.500.9001.0
161     11.20.27.813.3015.2471.33903900.0.1.500.7028.1
2019    19.10.27.812.3017.4503.33503900.0.1.501.7015.1
5552    56.10.15.452.3022.2341.33903000.0.1.500.9001.0
4524     38.10.6.122.3024.2100.33909300.0.1.500.9001.1
3212    25.10.13.392.3001.6420.33903600.0.1.500.9001.0
1090    16.12.12.365.3025.4360.33903900.0.1.500.9001.1
4365    34.10.14.422.3023.4321.33903300.0.1.500.9001.0
Name: dotacao, dtype: object

In [25]:
df_execucao['dotacao'].duplicated().any()

np.False_

In [26]:
df_execucao['dotacao'].duplicated().mean()

np.float64(0.0)

In [27]:
df_execucao['dotacao'].str.split('.').apply(len).unique()

array([12])

In [28]:
colunas_descricao = [col for col in df_execucao.columns
                     if col.startswith('Ds') or col.startswith('TXT')
                     and col not in ('Ds_Funcao', 
                     'Ds_SubFuncao',
                     'Ds_Programa',
                     'Ds_Despesa', 
                     'Ds_Categoria', 
                     'Ds_Grupo', 
                     'Ds_Modalidade',
                     'TXT_FONT_REC_RDZD')]

In [29]:
colunas_descricao

['Ds_Orgao',
 'Ds_Unidade',
 'Ds_Funcao',
 'Ds_SubFuncao',
 'Ds_Programa',
 'Ds_Projeto_Atividade',
 'Ds_Despesa',
 'Ds_Categoria',
 'Ds_Grupo',
 'Ds_Modalidade',
 'Ds_Fonte',
 'TXT_EX_FONT_REC',
 'TXT_DSTN_REC',
 'TXT_VINC_PMSP',
 'TXT_TIP_CRED_ORCM']

In [30]:
[col for col in df_execucao.columns if col not in 
 colunas_descricao and col not in codigos_dotacao]

['DataInicial',
 'DataFinal',
 'Cd_AnoExecucao',
 'Cd_Exercicio',
 'Cd_Dotacao_Id',
 'Administracao',
 'Sigla_Orgao',
 'PA',
 'PAPA',
 'ProjAtiv_Descricao',
 'Categoria_Despesa',
 'Grupo_Despesa',
 'Cd_Modalidade',
 'Cd_Elemento',
 'COD_RDZD_FONT_REC',
 'TXT_FONT_REC_RDZD',
 'Cd_Nro_Emenda_Dotacao',
 'Vl_Orcado_Ano',
 'Vl_Suplementado',
 'Vl_Reduzido',
 'Vl_SuplementadoLiquido',
 'Vl_SuplementadoEmTramitacao',
 'Vl_ReduzidoEmTramitacao',
 'Vl_Orcado_Atualizado',
 'Vl_Congelado',
 'Vl_Descongelado',
 'Vl_CongeladoLiquido',
 'Disponivel',
 'Vl_ReservadoLiquido',
 'Vl_EmpenhadoLiquido',
 'Vl_Liquidado',
 'Vl_Pago',
 'Saldo_Dotacao',
 'DataExtracao',
 'dotacao']

In [31]:
cols_valores: list[str] = ['Vl_Orcado_Ano',
                        'Vl_Orcado_Atualizado', 
                        'Vl_EmpenhadoLiquido', 
                        'Vl_CongeladoLiquido', 
                        'Vl_Liquidado', 
                        ]

In [32]:
col_id: list[str] = ['Cd_Dotacao_Id', 'dotacao', 'Cd_Nro_Emenda_Dotacao']

In [33]:
cols_interesse: list[str] = col_id + codigos_dotacao + colunas_descricao + cols_valores

In [34]:
df_execucao: DataFrame = df_execucao[cols_interesse]

In [35]:
df_execucao.head()

,Cd_Dotacao_Id,dotacao,Cd_Nro_Emenda_Dotacao,Cd_Orgao,Cd_Unidade,Cd_Funcao,Cd_SubFuncao,Cd_Programa,ProjetoAtividade,Cd_Despesa,Cd_Fonte,COD_EX_FONT_REC,COD_DSTN_REC,COD_VINC_REC_PMSP,COD_TIP_CRED_ORCM,Ds_Orgao,Ds_Unidade,Ds_Funcao,Ds_SubFuncao,Ds_Programa,Ds_Projeto_Atividade,Ds_Despesa,Ds_Categoria,Ds_Grupo,Ds_Modalidade,Ds_Fonte,TXT_EX_FONT_REC,TXT_DSTN_REC,TXT_VINC_PMSP,TXT_TIP_CRED_ORCM,Vl_Orcado_Ano,Vl_Orcado_Atualizado,Vl_EmpenhadoLiquido,Vl_CongeladoLiquido,Vl_Liquidado
0,169108,7.10.8.244.3023.3399.44905100.8.1.759.1224.0,NaN,7,10,8,244,3023,3399,44905100,8,1,759,1224,0,Fundo Municipal de Desenvolvimento Social,Fundo Municipal de Desenvolvimento Social,Assistência Social,Assistência Comunitária,Proteção à População em Situação de Vulnerabil...,"Ampliação, Reforma e Requalificação de Equipam...",Obras e Instalações,Despesas de Capital,INVESTIMENTOS,Aplicações Diretas,Tesouro Municipal - Recursos Vinculados,Exercício corrente,Recursos vinculados a fundos,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,Inicial,1000,1000,0,0,0
1,173723,7.10.10.301.3003.1525.44505200.8.1.759.1224.1,NaN,7,10,10,301,3003,1525,44505200,8,1,759,1224,1,Fundo Municipal de Desenvolvimento Social,Fundo Municipal de Desenvolvimento Social,Saúde,Atenção Básica,"Ações e Serviços da Saúde em Atenção Básica, E...",Construção e Implantação de Equipamentos de At...,Equipamentos e Material Permanente,Despesas de Capital,INVESTIMENTOS,Transferências a Instituições Privadas sem fin...,Tesouro Municipal - Recursos Vinculados,Exercício corrente,Recursos vinculados a fundos,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,Suplementar,0,"1317978,6",1314177,0,1314177
2,171046,7.10.10.301.3003.1525.44905100.8.1.759.1224.0,NaN,7,10,10,301,3003,1525,44905100,8,1,759,1224,0,Fundo Municipal de Desenvolvimento Social,Fundo Municipal de Desenvolvimento Social,Saúde,Atenção Básica,"Ações e Serviços da Saúde em Atenção Básica, E...",Construção e Implantação de Equipamentos de At...,Obras e Instalações,Despesas de Capital,INVESTIMENTOS,Aplicações Diretas,Tesouro Municipal - Recursos Vinculados,Exercício corrente,Recursos vinculados a fundos,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,Inicial,4449303,"3131324,4","3131324,4",0,"2940879,9"
3,173862,7.10.10.301.3003.1525.44905100.8.2.759.1224.1,NaN,7,10,10,301,3003,1525,44905100,8,2,759,1224,1,Fundo Municipal de Desenvolvimento Social,Fundo Municipal de Desenvolvimento Social,Saúde,Atenção Básica,"Ações e Serviços da Saúde em Atenção Básica, E...",Construção e Implantação de Equipamentos de At...,Obras e Instalações,Despesas de Capital,INVESTIMENTOS,Aplicações Diretas,Tesouro Municipal - Recursos Vinculados,Exercícios anteriores,Recursos vinculados a fundos,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,Suplementar,0,"37258939,15","37222552,12",0,"30999672,71"
4,180471,7.10.10.301.3003.1525.44905100.10.1.755.1224.1,NaN,7,10,10,301,3003,1525,44905100,10,1,755,1224,1,Fundo Municipal de Desenvolvimento Social,Fundo Municipal de Desenvolvimento Social,Saúde,Atenção Básica,"Ações e Serviços da Saúde em Atenção Básica, E...",Construção e Implantação de Equipamentos de At...,Obras e Instalações,Despesas de Capital,INVESTIMENTOS,Aplicações Diretas,Alienação de Bens/Ativos,Exercício corrente,Recursos de Alienação de Bens/Ativos - Adminis...,PMSP-SF/COADM-Fundo Municipal de Desenvolvimen...,Suplementar,0,2000000,2000000,0,2000000


In [36]:
df_execucao['Cd_Dotacao_Id'].duplicated().any()

np.False_

In [37]:
df_execucao['dotacao'].duplicated().any()

np.False_

Agora vamos limpar o df de regionalizacao

In [38]:
df_regionalizacao.head()

,COD_EMPRESA_PMSP,COD_EMPENHO,ANO_EMPENHO,CÓDIGO_NLP,ANO_LIQUIDAÇÃO,DATA_LIQUIDAÇÃO,CÓDIGO_ÓRGÃO,SIGLA_ÓRGÃO,DESCRIÇÃO_ÓRGÃO,CÓDIGO_UNIDADE,DESCRIÇÃO_UNIDADE,CÓDIGO_FUNÇÃO,DESCRIÇÃO_FUNÇÃO,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,CÓDIGO_PROJ_ATIV,DESCRIÇÃO_PROJ_ATIV,CÓDIGO_CONTA_DESPESA,DESCRIÇÃO_CONTA_DESPESA,CÓDIGO_FONTE,DESCRIÇÃO_FONTE,CÓDIGO_EXERCÍCIO_FONTE,CÓDIGO_DESTINAÇÃO_RECURSO,CÓDIGO_VÍNCULO_PMSP,CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,DATA_EXTRAÇÃO
0,4,197,2024,362,2024,2024-01-30 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3024,Suporte Administrativo,2100,Administração da Unidade,31900700,Contribuição a Entidades Fechadas de Previdência,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1792,03",15/03/2025
1,4,197,2024,893,2024,2024-02-28 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3024,Suporte Administrativo,2100,Administração da Unidade,31900700,Contribuição a Entidades Fechadas de Previdência,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1780,47",15/03/2025
2,4,197,2024,1973,2024,2024-04-05 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3024,Suporte Administrativo,2100,Administração da Unidade,31900700,Contribuição a Entidades Fechadas de Previdência,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1667,19",15/03/2025
3,4,197,2024,2430,2024,2024-04-29 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3024,Suporte Administrativo,2100,Administração da Unidade,31900700,Contribuição a Entidades Fechadas de Previdência,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1667,19",15/03/2025
4,4,2028,2024,3142,2024,2024-05-29 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,Hospital do Servidor Público Municipal,10,Saúde,122,Administração Geral,3024,Suporte Administrativo,2100,Administração da Unidade,31900700,Contribuição a Entidades Fechadas de Previdência,0,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,"1782,02",15/03/2025


In [39]:
df_regionalizacao['COD_EMPRESA_PMSP'].unique()

array([ 4,  5,  8,  9, 30, 29,  1, 18, 19,  3,  7, 22, 23, 16, 81, 83, 85,
       91])

In [40]:
#filtrando para administração direta

df_regionalizacao = df_regionalizacao[df_regionalizacao['COD_EMPRESA_PMSP']==1]

In [41]:
#filtrando para garantir que a liquidacao é de 2024

In [42]:
df_regionalizacao['ANO_LIQUIDAÇÃO'].unique()

array([2024, 2025])

In [43]:
df_regionalizacao = df_regionalizacao[df_regionalizacao['ANO_LIQUIDAÇÃO']==2024].reset_index().copy()

In [44]:
df_regionalizacao.columns

Index(['index', 'COD_EMPRESA_PMSP', 'COD_EMPENHO', 'ANO_EMPENHO', 'CÓDIGO_NLP',
       'ANO_LIQUIDAÇÃO', 'DATA_LIQUIDAÇÃO', 'CÓDIGO_ÓRGÃO', 'SIGLA_ÓRGÃO',
       'DESCRIÇÃO_ÓRGÃO', 'CÓDIGO_UNIDADE', 'DESCRIÇÃO_UNIDADE',
       'CÓDIGO_FUNÇÃO', 'DESCRIÇÃO_FUNÇÃO', 'CÓDIGO_SUBFUNÇÃO',
       'DESCRIÇÃO_SUBFUNÇÃO', 'CÓDIGO_PROGRAMA', 'DESCRIÇÃO_PROGRAMA',
       'CÓDIGO_PROJ_ATIV', 'DESCRIÇÃO_PROJ_ATIV', 'CÓDIGO_CONTA_DESPESA',
       'DESCRIÇÃO_CONTA_DESPESA', 'CÓDIGO_FONTE', 'DESCRIÇÃO_FONTE',
       'CÓDIGO_EXERCÍCIO_FONTE', 'CÓDIGO_DESTINAÇÃO_RECURSO',
       'CÓDIGO_VÍNCULO_PMSP', 'CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO', 'REGIÃO',
       'SUBPREFEITURA', 'DISTRITO', 'TIPO_REGIONALIZAÇÃO',
       'VALOR_DETALHAMENTO_AÇÃO', 'DATA_EXTRAÇÃO'],
      dtype='object')

In [45]:
cols_cod = [col for col in df_regionalizacao.columns if col.lower().startswith('cod') or col.lower().startswith('cód')]

In [46]:
cols_cod

['COD_EMPRESA_PMSP',
 'COD_EMPENHO',
 'CÓDIGO_NLP',
 'CÓDIGO_ÓRGÃO',
 'CÓDIGO_UNIDADE',
 'CÓDIGO_FUNÇÃO',
 'CÓDIGO_SUBFUNÇÃO',
 'CÓDIGO_PROGRAMA',
 'CÓDIGO_PROJ_ATIV',
 'CÓDIGO_CONTA_DESPESA',
 'CÓDIGO_FONTE',
 'CÓDIGO_EXERCÍCIO_FONTE',
 'CÓDIGO_DESTINAÇÃO_RECURSO',
 'CÓDIGO_VÍNCULO_PMSP',
 'CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO']

In [47]:
#printando os códigos da dotação do dataframe anterior para comparar
codigos_dotacao

['Cd_Orgao',
 'Cd_Unidade',
 'Cd_Funcao',
 'Cd_SubFuncao',
 'Cd_Programa',
 'ProjetoAtividade',
 'Cd_Despesa',
 'Cd_Fonte',
 'COD_EX_FONT_REC',
 'COD_DSTN_REC',
 'COD_VINC_REC_PMSP',
 'COD_TIP_CRED_ORCM']

In [48]:
cols_dotacao_regionaliz = [
'CÓDIGO_ÓRGÃO',
'CÓDIGO_UNIDADE',
'CÓDIGO_FUNÇÃO',
'CÓDIGO_SUBFUNÇÃO',
'CÓDIGO_PROGRAMA',
 'CÓDIGO_PROJ_ATIV',
 'CÓDIGO_CONTA_DESPESA',
 'CÓDIGO_FONTE',
 'CÓDIGO_EXERCÍCIO_FONTE',
 'CÓDIGO_DESTINAÇÃO_RECURSO',
 'CÓDIGO_VÍNCULO_PMSP',
 'CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO'
]

In [49]:
for col in cols_dotacao_regionaliz:
    df_regionalizacao[col] = df_regionalizacao[col].astype(int)

In [50]:
df_regionalizacao['dotacao'] = df_regionalizacao[cols_dotacao_regionaliz].astype(str).agg('.'.join, axis=1)

In [51]:
df_regionalizacao['dotacao'].str.split('.').apply(len).unique()

array([12])

In [52]:
#aqui posssui duplicacao para dotacao porque a dotacao pode ter regionalizacoes diferentes

df_regionalizacao['dotacao'].duplicated().mean()

np.float64(0.9900482749962909)

In [53]:
df_regionalizacao.columns

Index(['index', 'COD_EMPRESA_PMSP', 'COD_EMPENHO', 'ANO_EMPENHO', 'CÓDIGO_NLP',
       'ANO_LIQUIDAÇÃO', 'DATA_LIQUIDAÇÃO', 'CÓDIGO_ÓRGÃO', 'SIGLA_ÓRGÃO',
       'DESCRIÇÃO_ÓRGÃO', 'CÓDIGO_UNIDADE', 'DESCRIÇÃO_UNIDADE',
       'CÓDIGO_FUNÇÃO', 'DESCRIÇÃO_FUNÇÃO', 'CÓDIGO_SUBFUNÇÃO',
       'DESCRIÇÃO_SUBFUNÇÃO', 'CÓDIGO_PROGRAMA', 'DESCRIÇÃO_PROGRAMA',
       'CÓDIGO_PROJ_ATIV', 'DESCRIÇÃO_PROJ_ATIV', 'CÓDIGO_CONTA_DESPESA',
       'DESCRIÇÃO_CONTA_DESPESA', 'CÓDIGO_FONTE', 'DESCRIÇÃO_FONTE',
       'CÓDIGO_EXERCÍCIO_FONTE', 'CÓDIGO_DESTINAÇÃO_RECURSO',
       'CÓDIGO_VÍNCULO_PMSP', 'CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO', 'REGIÃO',
       'SUBPREFEITURA', 'DISTRITO', 'TIPO_REGIONALIZAÇÃO',
       'VALOR_DETALHAMENTO_AÇÃO', 'DATA_EXTRAÇÃO', 'dotacao'],
      dtype='object')

In [54]:
cols_regioalizacao = ['dotacao', 'REGIÃO', 'SUBPREFEITURA', 'DISTRITO', 'TIPO_REGIONALIZAÇÃO', 'VALOR_DETALHAMENTO_AÇÃO']

In [55]:
df_regionalizacao = df_regionalizacao[cols_regioalizacao].copy(deep=True)

### Agora vamos dar o join

In [56]:
df_merge = pd.merge(df_execucao, df_regionalizacao, how='outer', on='dotacao')

In [57]:
#checando se nao ha regionalizacao sem dotacao (nao deveria haver)

assert not df_merge['Cd_Dotacao_Id'].isnull().any()

Vamos criar a variável tratamento - identifica se é emenda

In [58]:
df_merge['Cd_Nro_Emenda_Dotacao'].isnull().mean()

np.float64(0.9992491779403051)

In [59]:
df_merge['is_emenda'] = df_merge['Cd_Nro_Emenda_Dotacao'].notnull().astype(int)

In [60]:
df_merge[df_merge['is_emenda'].astype(bool)].head()

,Cd_Dotacao_Id,dotacao,Cd_Nro_Emenda_Dotacao,Cd_Orgao,Cd_Unidade,Cd_Funcao,Cd_SubFuncao,Cd_Programa,ProjetoAtividade,Cd_Despesa,Cd_Fonte,COD_EX_FONT_REC,COD_DSTN_REC,COD_VINC_REC_PMSP,COD_TIP_CRED_ORCM,Ds_Orgao,Ds_Unidade,Ds_Funcao,Ds_SubFuncao,Ds_Programa,Ds_Projeto_Atividade,Ds_Despesa,Ds_Categoria,Ds_Grupo,Ds_Modalidade,Ds_Fonte,TXT_EX_FONT_REC,TXT_DSTN_REC,TXT_VINC_PMSP,TXT_TIP_CRED_ORCM,Vl_Orcado_Ano,Vl_Orcado_Atualizado,Vl_EmpenhadoLiquido,Vl_CongeladoLiquido,Vl_Liquidado,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,is_emenda
899,172670,11.20.17.451.3005.8032.33903900.7.9.799.0.0,6486.0,11,20,17,451,3005,8032,33903900,7,9,799,0,0,Secretaria do Governo Municipal,Gabinete do Secretário,Saneamento,Infra-Estrutura Urbana,Promoção da Sustentabilidade Ambiental,Operação e Manutenção de Serviços de Saneament...,Outros Serviços de Terceiros - Pessoa Jurídica,Despesas Correntes,OUTRAS DESPESAS CORRENTES,Aplicações Diretas,Receita Condicionada,Exercícios condicionados,Outras vinculações legais,Código Genérico,Inicial,1000,1000,0,1000,0,NaN,NaN,NaN,NaN,NaN,1
3271,172676,11.60.4.122.3015.8046.33903900.0.1.500.9001.0,2078.0,11,60,4,122,3015,8046,33903900,0,1,500,9001,0,Secretaria do Governo Municipal,Casa Civil,Administração,Administração Geral,Promoção da Cidade como Referência Global e De...,Estruturas de Eventos 2024,Outros Serviços de Terceiros - Pessoa Jurídica,Despesas Correntes,OUTRAS DESPESAS CORRENTES,Aplicações Diretas,Tesouro Municipal,Exercício corrente,Recursos não vinculados de Impostos,Sem detalhamento,Inicial,700000,0,0,0,0,NaN,NaN,NaN,NaN,NaN,1
3279,172678,11.60.4.122.3024.8043.33903900.0.1.500.9001.0,1230.0,11,60,4,122,3024,8043,33903900,0,1,500,9001,0,Secretaria do Governo Municipal,Casa Civil,Administração,Administração Geral,Suporte Administrativo,Ações Voltadas para Políticas Públicas,Outros Serviços de Terceiros - Pessoa Jurídica,Despesas Correntes,OUTRAS DESPESAS CORRENTES,Aplicações Diretas,Tesouro Municipal,Exercício corrente,Recursos não vinculados de Impostos,Sem detalhamento,Inicial,5000000,0,0,0,0,NaN,NaN,NaN,NaN,NaN,1
3280,172679,11.60.4.122.3024.8044.33903900.0.1.500.9001.0,477.0,11,60,4,122,3024,8044,33903900,0,1,500,9001,0,Secretaria do Governo Municipal,Casa Civil,Administração,Administração Geral,Suporte Administrativo,Ações voltadas para Políticas Públicas,Outros Serviços de Terceiros - Pessoa Jurídica,Despesas Correntes,OUTRAS DESPESAS CORRENTES,Aplicações Diretas,Tesouro Municipal,Exercício corrente,Recursos não vinculados de Impostos,Sem detalhamento,Inicial,5000000,0,0,0,0,NaN,NaN,NaN,NaN,NaN,1
3281,172680,11.60.4.122.3024.8045.33903900.0.1.500.9001.0,2478.0,11,60,4,122,3024,8045,33903900,0,1,500,9001,0,Secretaria do Governo Municipal,Casa Civil,Administração,Administração Geral,Suporte Administrativo,Ações voltadas para Políticas Públicas.,Outros Serviços de Terceiros - Pessoa Jurídica,Despesas Correntes,OUTRAS DESPESAS CORRENTES,Aplicações Diretas,Tesouro Municipal,Exercício corrente,Recursos não vinculados de Impostos,Sem detalhamento,Inicial,5000000,0,0,0,0,NaN,NaN,NaN,NaN,NaN,1


In [61]:
df_merge.columns

Index(['Cd_Dotacao_Id', 'dotacao', 'Cd_Nro_Emenda_Dotacao', 'Cd_Orgao',
       'Cd_Unidade', 'Cd_Funcao', 'Cd_SubFuncao', 'Cd_Programa',
       'ProjetoAtividade', 'Cd_Despesa', 'Cd_Fonte', 'COD_EX_FONT_REC',
       'COD_DSTN_REC', 'COD_VINC_REC_PMSP', 'COD_TIP_CRED_ORCM', 'Ds_Orgao',
       'Ds_Unidade', 'Ds_Funcao', 'Ds_SubFuncao', 'Ds_Programa',
       'Ds_Projeto_Atividade', 'Ds_Despesa', 'Ds_Categoria', 'Ds_Grupo',
       'Ds_Modalidade', 'Ds_Fonte', 'TXT_EX_FONT_REC', 'TXT_DSTN_REC',
       'TXT_VINC_PMSP', 'TXT_TIP_CRED_ORCM', 'Vl_Orcado_Ano',
       'Vl_Orcado_Atualizado', 'Vl_EmpenhadoLiquido', 'Vl_CongeladoLiquido',
       'Vl_Liquidado', 'REGIÃO', 'SUBPREFEITURA', 'DISTRITO',
       'TIPO_REGIONALIZAÇÃO', 'VALOR_DETALHAMENTO_AÇÃO', 'is_emenda'],
      dtype='object')

In [62]:

colunas = {
    'Cd_Dotacao_Id' : 'id_dotacao',
    'dotacao' : 'dotacao',
    'is_emenda' : 'is_emenda',
    'REGIÃO' : 'regiao',
    'SUBPREFEITURA' : 'subprefeitura',
    'DISTRITO' : 'distrito',
    'TIPO_REGIONALIZAÇÃO' : 'tipo_regionalizacao',
    'VALOR_DETALHAMENTO_AÇÃO' : 'val_liquidado_regionalizado',
    'Vl_Orcado_Ano' : 'val_orcado_dotacao', 
    'Vl_Orcado_Atualizado' : 'val_orcado_atualiz_dotacao',
    'Vl_EmpenhadoLiquido' : 'val_empenhado_liquido_dotacao', 
    'Vl_CongeladoLiquido' : 'val_congelado_liquido_dotacao', 
    'Vl_Liquidado' : 'val_liquidado_dotacao',
   'Cd_Orgao' : 'cd_orgao',
   'Ds_Orgao' : 'desc_orgao',
   'Cd_Unidade' : 'cd_unidade',
   'Ds_Unidade' : 'desc_unidade',
   'Cd_Funcao' : 'cd_funcao',
   'Cd_SubFuncao' : 'cd_subfuncao', 
   'Ds_SubFuncao' : 'desc_subfuncao',
    'Cd_Programa' : 'cd_programa',
    'Ds_Programa' : 'desc_programa',
    'ProjetoAtividade' : 'cd_proj_ativ', 
    'Ds_Projeto_Atividade' : 'desc_proj_ativ',
    'Cd_Despesa' : 'cd_despesa', 
    'Ds_Despesa' : 'desc_despesa',
    'Cd_Fonte' : 'cd_fonte', 
    'Ds_Fonte' : 'ds_fonte',
    'COD_EX_FONT_REC' : 'cod_exerc_fonte_recurso',
    'TXT_EX_FONT_REC' : 'desc_exerc_fonte_recurso',
    'COD_DSTN_REC' : 'cod_destinacao_recurso', 
    'TXT_DSTN_REC' : 'desc_destinacao_recurso',
    'COD_VINC_REC_PMSP' : 'cod_vinculacao_recurso',
    'TXT_VINC_PMSP' : 'desc_vinculacao_recurso',
    'COD_TIP_CRED_ORCM' : 'cod_tipo_credito_orcamentario',
    'TXT_TIP_CRED_ORCM' : 'desc_tipo_credito_orcamentario'

}

In [63]:
df_merge = df_merge[list(colunas.keys())].copy(deep=True)

In [64]:
df_merge.rename(colunas, axis=1, inplace=True)

In [65]:
df_merge.sample(3)

,id_dotacao,dotacao,is_emenda,regiao,subprefeitura,distrito,tipo_regionalizacao,val_liquidado_regionalizado,val_orcado_dotacao,val_orcado_atualiz_dotacao,val_empenhado_liquido_dotacao,val_congelado_liquido_dotacao,val_liquidado_dotacao,cd_orgao,desc_orgao,cd_unidade,desc_unidade,cd_funcao,cd_subfuncao,desc_subfuncao,cd_programa,desc_programa,cd_proj_ativ,desc_proj_ativ,cd_despesa,desc_despesa,cd_fonte,ds_fonte,cod_exerc_fonte_recurso,desc_exerc_fonte_recurso,cod_destinacao_recurso,desc_destinacao_recurso,cod_vinculacao_recurso,desc_vinculacao_recurso,cod_tipo_credito_orcamentario,desc_tipo_credito_orcamentario
233189,170720,27.10.18.541.3005.6669.33903600.0.1.500.9001.0,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,460,559317,389903,352705,34093,306015,27,Secretaria Municipal do Verde e do Meio Ambiente,10,Gabinete do Secretário,18,541,Preservação e Conservação Ambiental,3005,Promoção da Sustentabilidade Ambiental,6669,Educação Ambiental,33903600,Outros Serviços de Terceiros - Pessoa Física,0,Tesouro Municipal,1,Exercício corrente,500,Recursos não vinculados de Impostos,9001,Sem detalhamento,0,Inicial
151446,168844,16.20.12.365.3025.2828.33503900.0.1.500.9001.0,0,Leste,Subprefeitura Itaim Paulista,Supra-Distrital,Despesa Regionalizável,"181,06",350231519,350231519,"350217186,1","11248,62","350202422,8",16,Secretaria Municipal de Educação,20,Diretoria Regional de Educação São Miguel,12,365,Educação Infantil,3025,Proteção e Desenvolvimento Integral na Primeir...,2828,Manutenção e Operação da Rede Parceira - Centr...,33503900,Outros Serviços de Terceiros - Pessoa Jurídica,0,Tesouro Municipal,1,Exercício corrente,500,Recursos não vinculados de Impostos,9001,Sem detalhamento,0,Inicial
223809,165450,25.10.13.392.3001.6394.33903600.0.1.500.9001.0,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,1500,1350000,1350000,774800,566400,770500,25,Secretaria Municipal de Cultura,10,Gabinete do Secretário,13,392,Difusão Cultural,3001,Acesso à Cultura,6394,Mês do Hip Hop,33903600,Outros Serviços de Terceiros - Pessoa Física,0,Tesouro Municipal,1,Exercício corrente,500,Recursos não vinculados de Impostos,9001,Sem detalhamento,0,Inicial


In [66]:
df_merge['is_emenda'].sum()

np.int64(332)

In [67]:
df_merge[df_merge['is_emenda'].astype(bool)]['dotacao'].nunique()

330

In [68]:
df_merge['regiao'].unique()

array([nan, 'Supra-Regional', 'Sul', 'Centro', 'Leste', 'Norte', 'Oeste'],
      dtype=object)

In [69]:
df_merge['regiao'].isnull().mean()

np.float64(0.10790127142217458)

In [70]:
len(df_merge)

442182

In [71]:
df_merge.columns

Index(['id_dotacao', 'dotacao', 'is_emenda', 'regiao', 'subprefeitura',
       'distrito', 'tipo_regionalizacao', 'val_liquidado_regionalizado',
       'val_orcado_dotacao', 'val_orcado_atualiz_dotacao',
       'val_empenhado_liquido_dotacao', 'val_congelado_liquido_dotacao',
       'val_liquidado_dotacao', 'cd_orgao', 'desc_orgao', 'cd_unidade',
       'desc_unidade', 'cd_funcao', 'cd_subfuncao', 'desc_subfuncao',
       'cd_programa', 'desc_programa', 'cd_proj_ativ', 'desc_proj_ativ',
       'cd_despesa', 'desc_despesa', 'cd_fonte', 'ds_fonte',
       'cod_exerc_fonte_recurso', 'desc_exerc_fonte_recurso',
       'cod_destinacao_recurso', 'desc_destinacao_recurso',
       'cod_vinculacao_recurso', 'desc_vinculacao_recurso',
       'cod_tipo_credito_orcamentario', 'desc_tipo_credito_orcamentario'],
      dtype='object')

In [72]:
df_merge.dtypes

id_dotacao                         int64
dotacao                           object
is_emenda                          int64
regiao                            object
subprefeitura                     object
distrito                          object
tipo_regionalizacao               object
val_liquidado_regionalizado       object
val_orcado_dotacao                 int64
val_orcado_atualiz_dotacao        object
val_empenhado_liquido_dotacao     object
val_congelado_liquido_dotacao     object
val_liquidado_dotacao             object
cd_orgao                           int64
desc_orgao                        object
cd_unidade                         int64
desc_unidade                      object
cd_funcao                          int64
cd_subfuncao                       int64
desc_subfuncao                    object
cd_programa                        int64
desc_programa                     object
cd_proj_ativ                       int64
desc_proj_ativ                    object
cd_despesa      

In [73]:
for col in df_merge.columns:
    if col.startswith('val'):
        df_merge[col] = df_merge[col].astype(str).str.replace(',', '.').astype(float)

In [74]:
pd.set_option('display.max_rows', None)

In [75]:
df_merge[df_merge['dotacao']=='19.10.27.812.3017.9254.44903900.0.1.501.7032.1']

,id_dotacao,dotacao,is_emenda,regiao,subprefeitura,distrito,tipo_regionalizacao,val_liquidado_regionalizado,val_orcado_dotacao,val_orcado_atualiz_dotacao,val_empenhado_liquido_dotacao,val_congelado_liquido_dotacao,val_liquidado_dotacao,cd_orgao,desc_orgao,cd_unidade,desc_unidade,cd_funcao,cd_subfuncao,desc_subfuncao,cd_programa,desc_programa,cd_proj_ativ,desc_proj_ativ,cd_despesa,desc_despesa,cd_fonte,ds_fonte,cod_exerc_fonte_recurso,desc_exerc_fonte_recurso,cod_destinacao_recurso,desc_destinacao_recurso,cod_vinculacao_recurso,desc_vinculacao_recurso,cod_tipo_credito_orcamentario,desc_tipo_credito_orcamentario
183289,174806,19.10.27.812.3017.9254.44903900.0.1.501.7032.1,1,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,216297.90,0.0,330000.0,330000.0,0.0,330000.0,19,Secretaria Municipal de Esportes e Lazer,10,Gabinete do Secretário,27,812,Desporto Comunitário,3017,"Promoção de Atividades Esportivas, Recreativas...",9254,Revitalização e Manutenção CDC Flor de Maio Es...,44903900,Outros Serviços de Terceiros - Pessoa Jurídica,0,Tesouro Municipal,1,Exercício corrente,501,Outros Recursos não Vinculados,7032,Emendas Parlamentares - Edir Sales,1,Suplementar
183290,174806,19.10.27.812.3017.9254.44903900.0.1.501.7032.1,1,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,56851.05,0.0,330000.0,330000.0,0.0,330000.0,19,Secretaria Municipal de Esportes e Lazer,10,Gabinete do Secretário,27,812,Desporto Comunitário,3017,"Promoção de Atividades Esportivas, Recreativas...",9254,Revitalização e Manutenção CDC Flor de Maio Es...,44903900,Outros Serviços de Terceiros - Pessoa Jurídica,0,Tesouro Municipal,1,Exercício corrente,501,Outros Recursos não Vinculados,7032,Emendas Parlamentares - Edir Sales,1,Suplementar
183291,174806,19.10.27.812.3017.9254.44903900.0.1.501.7032.1,1,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,56851.05,0.0,330000.0,330000.0,0.0,330000.0,19,Secretaria Municipal de Esportes e Lazer,10,Gabinete do Secretário,27,812,Desporto Comunitário,3017,"Promoção de Atividades Esportivas, Recreativas...",9254,Revitalização e Manutenção CDC Flor de Maio Es...,44903900,Outros Serviços de Terceiros - Pessoa Jurídica,0,Tesouro Municipal,1,Exercício corrente,501,Outros Recursos não Vinculados,7032,Emendas Parlamentares - Edir Sales,1,Suplementar


In [76]:
df_merge[df_merge['dotacao']=='19.10.27.812.3017.9254.44903900.0.1.501.7032.1']['val_liquidado_regionalizado'].sum()

np.float64(330000.0)

In [77]:
liquidado_regionalizado = df_merge.groupby('dotacao')[['val_liquidado_regionalizado']].sum().reset_index()

In [78]:
liquidado_geral = df_merge.groupby('dotacao')[['val_liquidado_dotacao']].first().reset_index()

In [79]:
teste_liquidado = pd.merge(liquidado_regionalizado, liquidado_geral, on='dotacao', how='outer')

In [80]:
teste_liquidado['dotacao'].duplicated().any()

np.False_

In [81]:
teste_liquidado['teste'] = teste_liquidado['val_liquidado_regionalizado']>teste_liquidado['val_liquidado_dotacao']

In [82]:
teste_liquidado['teste'].any()

np.True_

In [83]:
teste_liquidado['teste'].mean()

np.float64(0.01685059926426961)

In [84]:
teste_liquidado[teste_liquidado['teste']].sample(10)

,dotacao,val_liquidado_regionalizado,val_liquidado_dotacao,teste
2331,22.10.15.126.3024.2171.33904000.0.1.500.9001.1,30078.73,30078.73,True
5045,49.10.15.452.3005.2705.33903900.0.1.500.9001.0,25577673.74,25577673.74,True
6241,67.10.15.451.3022.1170.44905100.0.1.500.9004.1,3659889.92,3659889.92,True
5205,52.10.15.122.3024.2100.31901100.0.1.500.9001.0,16573552.87,16573552.87,True
2717,25.10.13.391.3001.6417.33903000.0.1.500.9001.0,1596.30,1596.30,True
5086,50.10.15.122.3024.2100.33903900.0.1.500.9001.0,2073721.64,2073721.64,True
1370,16.17.12.367.3010.2827.33903900.0.1.500.9001.0,32281.09,32281.09,True
2175,20.50.26.453.3009.1099.44905100.2.2.700.885.1,2120419.76,2120419.76,True
368,13.10.4.122.3024.2100.33904600.0.1.500.9001.1,247477.29,247477.29,True
3866,30.10.12.366.3019.4434.33903900.0.1.500.9001.0,288401.92,288401.92,True


In [85]:
teste_liquidado['teste2'] = teste_liquidado['val_liquidado_regionalizado'] - teste_liquidado['val_liquidado_dotacao']

In [86]:
teste_liquidado[teste_liquidado['teste2']>0]['teste2'].mean()

np.float64(0.00514084586895621)

In [87]:
teste_liquidado[teste_liquidado['teste2']>0]['teste2'].max()

np.float64(0.1700000762939453)

In [88]:
df_merge['percentual_regionalizado'] = df_merge['val_liquidado_regionalizado']/df_merge['val_liquidado_dotacao']

In [89]:
temp = df_merge.groupby('is_emenda')[['val_liquidado_regionalizado', 'val_liquidado_dotacao']].sum()

In [90]:
temp = temp.reset_index(drop=True)

In [91]:
temp['percent_regionalizado'] = temp['val_liquidado_regionalizado']/temp['val_liquidado_dotacao']

In [92]:
temp

,val_liquidado_regionalizado,val_liquidado_dotacao,percent_regionalizado
0,7.288777e+10,4.669701e+13,0.001561
1,3.300000e+05,9.900000e+05,0.333333


In [93]:
df_merge.shape

(442182, 37)

In [95]:
df_merge.to_csv('data/dados_regionalizados_merge.csv', sep=';', index=False)